In [1]:
import pandas as pd
import sqlite3

In [2]:
# Create a sample DataFrame
data = {'name': ['Annie', 'Bob', 'Cindy', 'Dave', 'Elle', 'Frank', 'Gabby', 'Hank'], 
        'math_grades': [95, 85, 75, 73, 85, 79, 91, 55],
        'english_grades': [90, 88, 92, 70, 89, 85, 95, 85],
        'history_grades': [85, 95, 70, 78, 81, 82, 90, 70]}
df = pd.DataFrame(data)

In [3]:
df

,name,math_grades,english_grades,history_grades
0,Annie,95,90,85
1,Bob,85,88,95
2,Cindy,75,92,70
3,Dave,73,70,78
4,Elle,85,89,81
5,Frank,79,85,82
6,Gabby,91,95,90
7,Hank,55,85,70


## 1. Introduction to SQLite DB

SQLite is a C-language library that implements a small, fast, self-contained, high-reliability, full-featured, SQL database engine. SQLite is the most used database engine in the world. SQLite is built into all mobile phones and most computers and comes bundled inside countless other applications that people use every day.

https://www.sqlite.org/ 

Techniques shown here can be readily applied to most relational databases such as MySQL, Postgresql, AWS Redshift, GCP BigQuery (with some caveats), MS SQL Server, Oracle DBs and more.

In [4]:
# Connect to the SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('my_database.db')

# Verify: Go to the file system and check if the database was created

In [5]:
# Write the DataFrame to the SQLite database
# - 'name' specifies the table name
# - 'con' specifies the connection object
# - 'if_exists' specifies how to handle the table if it already exists:
# - 'fail': raise an error
# - 'replace': drop the table if it exists and create a new one
# - 'append': append to the existing table
# - 'index' specifies whether to write the DataFrame index as a column

df.to_sql(name='student_grades', con=conn, if_exists='replace', index=False)

# Commit the changes and close the connection
conn.commit()
conn.close()

In [6]:
# Example: Select data from the table

conn = sqlite3.connect('my_database.db')
cursor = conn.cursor()

cursor.execute("SELECT * FROM student_grades")

rows = cursor.fetchall()
for row in rows:
    print(row)

conn.close()

('Annie', 95, 90, 85)
('Bob', 85, 88, 95)
('Cindy', 75, 92, 70)
('Dave', 73, 70, 78)
('Elle', 85, 89, 81)
('Frank', 79, 85, 82)
('Gabby', 91, 95, 90)
('Hank', 55, 85, 70)


In [7]:
# Example 2: Filter

conn = sqlite3.connect('my_database.db')
cursor = conn.cursor()

cursor.execute('''
    SELECT * 
    FROM student_grades
    WHERE math_grades > 80''')

rows = cursor.fetchall()
for row in rows:
    print(row)

conn.close()

('Annie', 95, 90, 85)
('Bob', 85, 88, 95)
('Elle', 85, 89, 81)
('Gabby', 91, 95, 90)


In [8]:
# Example 3 : aggregation

conn = sqlite3.connect('my_database.db')
cursor = conn.cursor()

cursor.execute('''
    SELECT 'math' as subject, avg(math_grades)
    FROM student_grades''')

rows = cursor.fetchall()
for row in rows:
    print(row)

conn.close()

('math', 79.75)


In [9]:
# Example 3 : aggregation

conn = sqlite3.connect('my_database.db')
cursor = conn.cursor()

cursor.execute('''
  SELECT name, math_grades, avg_math_grades,
        CASE WHEN math_grades > avg_math_grades THEN 'above average'
              WHEN math_grades < avg_math_grades THEN 'below average'
              ELSE 'average' 
    END as grade_comparison
    FROM student_grades
  JOIN 
  (SELECT 'math' as subject, avg(math_grades) as avg_math_grades
  FROM student_grades) math 
  ''')

rows = cursor.fetchall()
for row in rows:
    print(row)

conn.close()

('Annie', 95, 79.75, 'above average')
('Bob', 85, 79.75, 'above average')
('Cindy', 75, 79.75, 'below average')
('Dave', 73, 79.75, 'below average')
('Elle', 85, 79.75, 'above average')
('Frank', 79, 79.75, 'below average')
('Gabby', 91, 79.75, 'above average')
('Hank', 55, 79.75, 'below average')


In [10]:
# Example 3 : aggregation

conn = sqlite3.connect('my_database.db')
cursor = conn.cursor()

query_stmt = '''
  SELECT name, math_grades, 
         CASE WHEN math_grades > avg_math_grades THEN '👍'
              WHEN math_grades < avg_math_grades THEN '👎'
              ELSE 'average' 
    END as grade_comparison
    FROM student_grades
  JOIN 
  (SELECT 'math' as subject, avg(math_grades) as avg_math_grades
  FROM student_grades) math 
  '''

cursor.execute(query_stmt)

rows = cursor.fetchall()
for row in rows:
    print(row)

conn.close()

('Annie', 95, '👍')
('Bob', 85, '👍')
('Cindy', 75, '👎')
('Dave', 73, '👎')
('Elle', 85, '👍')
('Frank', 79, '👎')
('Gabby', 91, '👍')
('Hank', 55, '👎')


In [11]:
# Example 3 : window functions (ranking and average)
# https://www.sqlitetutorial.net/sqlite-window-functions/

conn = sqlite3.connect('my_database.db')
cursor = conn.cursor()

cursor.execute('''
  SELECT name, math_grades, percent_rank() over (order by math_grades) as math_rank,
         avg(math_grades) over () as avg_math_grades
    FROM student_grades
  ''')

rows = cursor.fetchall()
for row in rows:
    print(row)

conn.close()

('Hank', 55, 0.0, 79.75)
('Dave', 73, 0.14285714285714285, 79.75)
('Cindy', 75, 0.2857142857142857, 79.75)
('Frank', 79, 0.42857142857142855, 79.75)
('Bob', 85, 0.5714285714285714, 79.75)
('Elle', 85, 0.5714285714285714, 79.75)
('Gabby', 91, 0.8571428571428571, 79.75)
('Annie', 95, 1.0, 79.75)


## Use Netflix Shows data

We can download free datasets from Kaggle.com, a data science and machine learning community.  Here's the Netflix data: https://www.kaggle.com/datasets/imtkaggleteam/netflix

In [12]:
# Connect to the SQLite database (or create it if it doesn't exist)
conn1 = sqlite3.connect('netflix.db')

# 1: Create a new dataframe from the NetFlix.csv file
netflix_df = pd.read_csv('data/NetFlix.csv')

# 2: Show the top 5 rows of the dataframe
netflix_df[:5]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,genres,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,14-Aug-20,2020,TV-MA,4,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s10,Movie,1920,Vikram Bhatt,"Rajneesh Duggal, Adah Sharma, Indraneil Sengup...",India,15-Dec-17,2008,TV-MA,143,"Horror Movies, International Movies, Thrillers",An architect and his wife move into a castle t...
2,s100,Movie,3 Heroines,Iman Brotoseno,"Reza Rahadian, Bunga Citra Lestari, Tara Basro...",Indonesia,5-Jan-19,2016,TV-PG,124,"Dramas, International Movies, Sports Movies",Three Indonesian women break records by becomi...
3,s1000,Movie,Blue Mountain State: The Rise of Thadland,Lev L. Spiro,"Alan Ritchson, Darin Brooks, James Cade, Rob R...",United States,1-Mar-16,2016,R,90,Comedies,New NFL star Thad buys his old teammates' belo...
4,s1001,TV Show,Blue Planet II,NaN,David Attenborough,United Kingdom,3-Dec-18,2017,TV-G,1,"British TV Shows, Docuseries, Science & Nature TV",This sequel to the award-winning nature series...


In [13]:
# 3: Save the DataFrame with the name 'netflix_shows' to a local SQLite database 

netflix_df.to_sql(name='netflix_shows', con=conn1, if_exists='replace', index=False)

7787

In [14]:
# 4: Get the data from the netflix table
conn1 = sqlite3.connect('netflix.db')
cursor1 = conn1.cursor()

cursor1.execute("SELECT * FROM netflix_shows limit 10")

rows = cursor1.fetchall()
for row in rows:
    print(row)

conn1.close()

('s1', 'TV Show', '3%', None, 'João Miguel, Bianca Comparato, Michel Gomes, Rodolfo Valente, Vaneza Oliveira, Rafael Lozano, Viviane Porto, Mel Fronckowiak, Sergio Mamberti, Zezé Motta, Celso Frateschi', 'Brazil', '14-Aug-20', 2020, 'TV-MA', 4, 'International TV Shows, TV Dramas, TV Sci-Fi & Fantasy', 'In a future where the elite inhabit an island paradise far from the crowded slums, you get one chance to join the 3% saved from squalor.')
('s10', 'Movie', '1920', 'Vikram Bhatt', 'Rajneesh Duggal, Adah Sharma, Indraneil Sengupta, Anjori Alagh, Rajendranath Zutshi, Vipin Sharma, Amin Hajee, Shri Vallabh Vyas', 'India', '15-Dec-17', 2008, 'TV-MA', 143, 'Horror Movies, International Movies, Thrillers', 'An architect and his wife move into a castle that is slated to become a luxury hotel. But something inside is determined to stop the renovation.')
('s100', 'Movie', '3 Heroines', 'Iman Brotoseno', 'Reza Rahadian, Bunga Citra Lestari, Tara Basro, Chelsea Islan', 'Indonesia', '5-Jan-19', 2016

In [15]:
# 5: Find all U.S. movies

conn1 = sqlite3.connect('netflix.db')
cursor1 = conn1.cursor()

cursor1.execute('''
    SELECT * 
    FROM netflix_shows 
    where country = \'United States\'
    ''')

rows = cursor1.fetchall()
for row in rows:
    print(row)

conn1.close()

('s1000', 'Movie', 'Blue Mountain State: The Rise of Thadland', 'Lev L. Spiro', 'Alan Ritchson, Darin Brooks, James Cade, Rob Ramsay, Chris Romano, Frankie Shaw, Omari Newton, Ed Marinaro, Dhani Jones, Ed Amatrudo, Jimmy Tatro', 'United States', '1-Mar-16', 2016, 'R', 90, 'Comedies', "New NFL star Thad buys his old teammates' beloved frat house, renames it Thadland and throws the raunchiest, most debauched party in school history.")
('s1004', 'Movie', 'Blue Valentine', 'Derek Cianfrance', 'Ryan Gosling, Michelle Williams, Faith Wladyka, John Doman, Mike Vogel, Ben Shenkman, Jen Jones, Maryann Plunkett, Marshall Johnson, James Benatti, Barbara Troy, Carey Westbrook, Enid Graham', 'United States', '5-Jul-18', 2010, 'R', 112, 'Dramas, Independent Movies, Romantic Movies', 'As Cindy and Dean muddle through their languishing marriage, they hearken back to the golden days when life was filled with possibility and romance.')
('s1009', 'Movie', 'Bo Burnham: Make Happy', 'Bo Burnham, Christophe

In [16]:
# 6: Find all U.S. horror titles (shows or movies)

conn1 = sqlite3.connect('netflix.db')
cursor1 = conn1.cursor()

query_stmt = '''
    SELECT * 
    FROM netflix_shows 
    where country = \'United States\'
    and (genres like \'%Horror%\' or genres like \'%horror%\')
    '''

cursor1.execute(query_stmt)

rows = cursor1.fetchall()
for row in rows:
    print(row)

conn1.close()

('s1162', 'Movie', 'Cabin Fever', 'Travis Zariwny', 'Gage Golightly, Matthew Daddario, Samuel Davis, Nadine Crocker, Dustin Ingram, Randy Schulman', 'United States', '6-Jun-19', 2016, 'R', 98, 'Horror Movies, Thrillers', "As a flesh-eating virus rips its way through a remote woodland cabin, the terrified teens vacationing inside wonder who'll fall victim next.")
('s1184', 'Movie', 'Cam', 'Daniel Goldhaber', 'Madeline Brewer, Patch Darragh, Melora Walters, Devin Druid, Imani Hakim, Michael Dempsey, Flora Diaz, Samantha Robinson, Jessica Parker Kennedy, Quei Tann', 'United States', '16-Nov-18', 2018, 'TV-MA', 95, 'Horror Movies, Independent Movies, Thrillers', 'After a look-alike takes over her account, a cam girl with a growing fan base sets out to identify the mysterious culprit and reclaim her own identity.')
('s1227', 'Movie', 'Carrie', 'Brian De Palma', 'Sissy Spacek, Piper Laurie, Amy Irving, William Katt, John Travolta, Nancy Allen, Betty Buckley, P.J. Soles, Priscilla Pointer, Sy

In [17]:
netflix_df.columns

Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'genres', 'description'],
      dtype='object')

In [18]:
# 6: Find all U.S. horror titles with the title or description with the word zombie
# 7: Convert Sqlite results back to a dataframe

conn1 = sqlite3.connect('netflix.db')
cursor1 = conn1.cursor()

query_stmt = '''
SELECT *
FROM netflix_shows 
WHERE country = \'United States\'
AND (genres like \'%Horror%\' or genres like \'%horror%\')
AND ((title like \'%Zombie%\' or title like \'%zombie%\') or 
        (description like \'%Zombie%\' or description like \'%zombie%\'))
'''

cursor1.execute(query_stmt)

rows = cursor1.fetchall()
for row in rows:
    print(row)

# Create Pandas DataFrame called df_new
df_new = pd.DataFrame(rows, columns=netflix_df.columns)

conn1.close()

('s2705', 'Movie', 'Here Alone', 'Rod Blackhurst', 'Lucy Walters, Gina Piersanti, Adam David Thompson, Shane West, Ryken A. Whitfield, Abel McSurely Bradshaw, Rebecca Spiro', 'United States', '1-Jul-17', 2016, 'TV-MA', 97, 'Horror Movies', 'When she finally encounters two other survivors, a woman alone in a world decimated by a zombie epidemic struggles to trust her new companions.')
('s3088', 'TV Show', 'IZombie', None, 'Rose McIver, Malcolm Goodwin, Rahul Kohli, Robert Buckley, David Anders, Aly Michalka', 'United States', '9-Aug-19', 2019, 'TV-MA', 5, 'TV Comedies, TV Dramas, TV Horror', 'A medical student-turned-zombie tries to retain her humanity by eating brains at the morgue and finds she has an uncanny new gift for solving crimes.')
('s6338', 'Movie', 'The Evil Dead', 'Sam Raimi', 'Bruce Campbell, Ellen Sandweiss, Richard DeManincor, Betsy Baker, Theresa Tilly, Bill Vincent, Scott Spiegel, Ivan Raimi, David Horton, John Cameron, Ted Raimi', 'United States', '10-Jan-20', 1981, '

In [19]:
df_new

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,genres,description
0,s2705,Movie,Here Alone,Rod Blackhurst,"Lucy Walters, Gina Piersanti, Adam David Thomp...",United States,1-Jul-17,2016,TV-MA,97,Horror Movies,When she finally encounters two other survivor...
1,s3088,TV Show,IZombie,None,"Rose McIver, Malcolm Goodwin, Rahul Kohli, Rob...",United States,9-Aug-19,2019,TV-MA,5,"TV Comedies, TV Dramas, TV Horror",A medical student-turned-zombie tries to retai...
2,s6338,Movie,The Evil Dead,Sam Raimi,"Bruce Campbell, Ellen Sandweiss, Richard DeMan...",United States,10-Jan-20,1981,TV-MA,85,"Classic Movies, Cult Movies, Horror Movies",During a stop at a remote cabin deep within th...
3,s7756,TV Show,Z Nation,None,"Nat Zang, Russell Hodgkinson, Michael Welch, A...",United States,27-Jan-19,2018,TV-MA,5,"TV Action & Adventure, TV Comedies, TV Horror",A team embarks on a perilous cross-country mis...
4,s7779,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,1-Nov-19,2009,R,88,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
